In [50]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression

mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()

images_test=np.array(images_test)
labels_test = np.array(labels_test)

images_train = np.array(images_train)
labels_train = np.array(labels_train)

solvers=["newton-cg","lbfgs","liblinear","sag"]
solvers=["liblinear"]


In [53]:
def classification_report_to_dataframe2(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        p = re.compile("avg / total")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [54]:
excel_file_name='Logistic' + str(datetime.now().strftime('%Y%m%d%H%m%S')) + '.xlsx'
df=pd.DataFrame(columns=['Solver','Accuracy','Normalized Accuracy',
                         'Time_taken_train','Time_taken_test','Negative Log loss',
                         'Macro_auc','Micro_auc'])

In [ ]:
for s in solvers:
    classifier=LogisticRegression(solver=s)
    start=time()
    classifier.fit(images_train,labels_train)
    train_time=time()-start
    
    start=time()
    predictions=classifier.predict(images_test)
    test_time=time()-start
    accuracy=accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction=classifier.predict_proba(images_test)
    log_loss_val=log_loss(labels_test,log_loss_prediction)
    classification_repo=classification_report(labels_test,predictions)
    
    labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")
    
    print(micro_auc,macro_auc)
    
    df_data=[s,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
    df.loc[len(df)]=df_data
    
excel_writer=pd.ExcelWriter(excel_file_name,engine='xlsxwriter')
df.to_excel(excel_writer, sheet_name='logistic')
df_classification_report=classification_report_to_dataframe2(classification_repo)
df_classification_report.to_excel(excel_writer, sheet_name='logistic_cr')
excel_writer.save()
    